In [1]:
#Load the librarys
import pandas as pd #To work with dataset
import numpy as np #Math library
import seaborn as sns #Graph library that use matplot in background
import matplotlib.pyplot as plt #to plot some parameters in seaborn
from IPython.display import display, Markdown, Latex

import plotly.offline as py # Permet de générer des graphiques hors ligne et de les enregistrer sur une machine locale
py.init_notebook_mode(connected=True) # Permet de travailler avec la version hors ligne de plotly
import plotly.graph_objs as go # C'est comme "plt" de matplot
import plotly.tools as tls # Il est utile d'obtenir des outils de plotly.
import warnings # Ignorer certains avertissements.
from collections import Counter # Pour faire le compteur de certaines fonctionnalités

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')
cmap=sns.color_palette('Blues_r')

In [83]:
#Importing the data
df = pd.read_excel("data\Entreprises.xlsx")
df.head(5)

,mark,pavitt,age,country,region,sector,pav,years,core_p,turnov,...,labour_flex,credit_req,credit_obt,fam_managed,fam_ceo,for_group,decentr_manag,bonus.1,qual_cert,comp
0,40,Specialized industries,More than 20 yrs,AUT,122,11,2,3,100.0,7.0,...,1,0,0,0,0,1,1,0,1,1
1,41,Traditional industries,More than 20 yrs,AUT,122,4,1,3,100.0,6.0,...,0,0,0,0,1,0,1,1,1,1
2,42,Economies of scale industries,More than 20 yrs,AUT,116,10,3,3,100.0,6.0,...,1,1,1,0,1,1,0,0,1,1
3,43,Traditional industries,Between 20 and 6 yrs,AUT,122,1,1,2,100.0,3.0,...,1,0,0,0,1,0,0,0,0,0
4,44,Traditional industries,More than 20 yrs,AUT,122,4,1,3,95.0,2.0,...,1,0,0,1,1,0,0,1,0,1


In [117]:
# Variables quantitatives
nb_nan = []
# nb_distincts =[]
for v in df.columns.tolist():
    nb_nan.append(df[v].isna().sum())
    # nb_distincts.append(len(set(df[v])))
    
df2 = pd.DataFrame()
df2 ["variables"] = df.columns.tolist()
# df2["Nb value distinct"] = nb_distincts
df2["Nb value missing"] = nb_nan 
df2["%Modalité_missing"] = round(df2["Nb value missing"]/len(df)*100, 1)

missings = df2.loc[df2["%Modalité_missing"] < 30]
missings.sort_values(by='%Modalité_missing')

,variables,Nb value missing,%Modalité_missing
0,mark,0,0.0
420,e3_m_c4,1,0.0
419,e3_m_c3,1,0.0
418,e3_m_c2,1,0.0
417,e3_m_c1,1,0.0
...,...,...,...
52,foreign_empl_n,3496,23.7
61,training_type,3543,24.0
46,unskilled_n,3726,25.2
20,a16_2,3805,25.8


In [124]:
temp1 = missings.variables.tolist() 
temp2 = df.columns.tolist() 

df3 = df[list(set(temp2) - set(temp1))]
df3.sample(5)

,d17b_m_c2,d11_2,d23c,f24_1,d12_3,d53_5,d17b_m_c3,f6_m_c2,d34_m_c4,d34_m_c2,...,f16_m_c5,d39_7,d39_3,d39_4,rd_financed_as_invest,d7_ric_m_c8,b9_2,e9,d67_spe_msf_12,f1_2
14365,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,...,NaN,NaN,NaN,NaN,2.0,1.0,NaN,2.0,NaN,NaN
11999,NaN,80.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
5534,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,1.0,...,0.0,NaN,NaN,NaN,2.0,0.0,NaN,1.0,NaN,NaN
11411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9291,NaN,2.0,1.0,NaN,2.0,NaN,NaN,NaN,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,80.0


In [129]:
print('Dim avant : ', df.shape)
print('Dim après : ', df3.shape)
print(df.shape[1] - df3.shape[1], "variables supprimés.")
print(temp1)

Dim avant :  (14759, 489)
Dim après :  (14759, 312)
177 variables supprimés.
['mark', 'pavitt', 'age', 'country', 'region', 'sector', 'pav', 'years', 'core_p', 'turnov', 'flop', 'group', 'acquire', 'acquired', 'affiliate', 'first_share', 'first_share_type', 'first_share_naz', 'a16_2', 'family', 'ceo_type', 'strategy', 'strategy_evolution', 'fin_benefit', 'non_fin_benefit', 'no_rewards', 'empl', 'no_family_p', 'no_family_n', 'family_p', 'family_n', 'family_tmt', 'white_n', 'skilled_n', 'unskilled_n', 'involv_p', 'involv_n', 'grad_p', 'grad_n', 'foreign_empl_n', 'fixed_contract', 'part_time', 'ceo_age', 'ceo_gender', 'abroad_exp_executives', 'training', 'training_type', 'trend_empl', 'adsl', 'c4_m_c1', 'c4_m_c2', 'c4_m_c3', 'c4_m_c4', 'c13a', 'prod_inn', 'proc_inn', 'no_inn', 'c17_m_c1', 'recent_patent', 'recent_ind_design', 'recent_trade_mark', 'c18_m_c1', 'c18_m_c2', 'c18_m_c3', 'c18_m_c4', 'rd_house_dummy', 'rd_acq_int_dummy', 'rd_acq_ext_dummy', 'rd_dummy', 'rd_self_fin', 'rd_intra_g

In [126]:
print(df3.columns.tolist())

['d17b_m_c2', 'd11_2', 'd23c', 'f24_1', 'd12_3', 'd53_5', 'd17b_m_c3', 'f6_m_c2', 'd34_m_c4', 'd34_m_c2', 'd39_2', 'd10_1', 'd53_6', 'f6_m_c4', 'd29_m_c5', 'f7', 'd46a_spe_msf_3', 'd29_m_c2', 'd67_spe_msf_1', 'b21aum', 'd67_spe_msf_17', 'f18', 'd55_m_c4', 'benefit_rd', 'd16', 'ln_k_l', 'd46a', 'f12_m_c2', 'd34_m_c8', 'd28_m_c7', 'd46b_spe_msf_1', 'd46a_spe_msf_17', 'exp_richness', 'c28_msf_6', 'f1_1', 'f16_m_c2', 'c16', 'rd_house', 'e14_m_c6', 'd7_3', 'e12', 'd36a', 'a18_3', 'd67_spe_msf_18', 'f6_m_c1', 'd67_spe_msf_15', 'f16_m_c1', 'd53_3', 'd50_8', 'd50_3', 'c28_msf_9', 'f16_m_c3', 'd56_m_c5', 'd46a_spe_msf_14', 'f12_m_c3', 'a16_3', 'd17a_m_c5', 'c28_msf_3', 'e13', 'd7_ric_m_c4', 'exp_15ue', 'd39_5', 'd46a_spe_msf_7', 'e14_m_c8', 'skilled_p', 'd55_m_c3', 'd53_8', 'd13_6', 'd46b_spe_msf_12', 'd46b_spe_msf_9', 'd7_1', 'd67', 'd20a', 'foreign_empl_p', 'd17a_m_c7', 'd46a_spe_msf_1', 'rd_ext', 'f12_m_c9', 'f12_m_c4', 'd50_5', 'a19_3', 'bonus', 'd46a_spe_msf_8', 'c28_msf_7', 'd38', 'd11_1'

In [130]:
df3.shape

(14759, 312)

In [135]:
df3.dtypes

d17b_m_c2         float64
d11_2             float64
d23c              float64
f24_1             float64
d12_3             float64
                   ...   
d7_ric_m_c8       float64
b9_2              float64
e9                float64
d67_spe_msf_12    float64
f1_2              float64
Length: 312, dtype: object

In [136]:
temp = df3
temp

,d17b_m_c2,d11_2,d23c,f24_1,d12_3,d53_5,d17b_m_c3,f6_m_c2,d34_m_c4,d34_m_c2,...,f16_m_c5,d39_7,d39_3,d39_4,rd_financed_as_invest,d7_ric_m_c8,b9_2,e9,d67_spe_msf_12,f1_2
0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,1.0,...,NaN,0.0,0.0,0.0,1.0,0.0,NaN,1.0,NaN,NaN
1,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,...,1.0,0.0,30.0,0.0,1.0,0.0,NaN,2.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,...,1.0,0.0,0.0,0.0,2.0,NaN,NaN,1.0,NaN,NaN
3,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
4,NaN,25.0,10.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,1.0,0.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14754,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN
14755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,2.0,NaN,NaN
14756,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2.0,NaN,NaN
14757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,2.0,NaN,NaN


In [113]:
df3

,d17b_m_c2,d11_2,d23c,f24_1,d12_3,d53_5,d17b_m_c3,f6_m_c2,d34_m_c4,d34_m_c2,...,f16_m_c5,d39_7,d39_3,d39_4,rd_financed_as_invest,d7_ric_m_c8,b9_2,e9,d67_spe_msf_12,f1_2
0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,1.0,...,NaN,0.0,0.0,0.0,1.0,0.0,NaN,1.0,NaN,NaN
1,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,...,1.0,0.0,30.0,0.0,1.0,0.0,NaN,2.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,...,1.0,0.0,0.0,0.0,2.0,NaN,NaN,1.0,NaN,NaN
3,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
4,NaN,25.0,10.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,1.0,0.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14754,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN
14755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,2.0,NaN,NaN
14756,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2.0,NaN,NaN
14757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,2.0,NaN,NaN


In [82]:
# Variables quantitatives
nb_nan = []
# nb_distincts =[]
for v in df.columns.tolist():
    nb_nan.append(df[v].isna().sum())
    # nb_distincts.append(len(set(df[v])))
    
df3 = pd.DataFrame()
df3 ["variables"] = df.columns.tolist()
# df3["Nb value distinct"] = nb_distincts
df3["Nb value missing"] = nb_nan 
df3["%Modalité_missing"] = round(df3["Nb value missing"]/len(df)*100, 1)

missings = df3.loc[df3["%Modalité_missing"]>0]
missings.sort_values(by='%Modalité_missing')

,variables,Nb value missing,%Modalité_missing
443,ea,12,0.1
10,flop,21,0.1
73,c4_m_c4,8,0.1
131,inn_postpone,9,0.1
27,family,13,0.1
...,...,...,...
290,d46b_spe_msf_11,14752,100.0
291,d46b_spe_msf_12,14752,100.0
294,d46b_spe_msf_15,14752,100.0
299,d46b_spe_msf_20,14752,100.0
